In [ ]:
# from transformers import AutoTokenizer, AutoModel
# from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd
import torch

In [ ]:
!pip install sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('nlpaueb/legal-bert-base-uncased')

In [ ]:
documents_df = pd.read_csv("/kaggle/input/preprocessed-data/AILA_Documents_Preprocessed_Data.csv")
queries_df = pd.read_csv("/kaggle/input/preprocessed-data/AILA_Queries_Preprocessed_Data.csv")
query_document_relevance_df = pd.read_csv("/kaggle/input/preprocessed-data/AILA_Relevance_Preprocessed_Data.csv")

In [ ]:
documents_df.head(5)
documents_df.shape

In [ ]:
queries_df.head(5)

In [ ]:
query_document_relevance_df.head(5)

In [ ]:
document_embeddings = model.encode(documents_df['Document_Text'].tolist(), convert_to_tensor=True)

In [ ]:
precision_scores = []
recall_scores = []

In [ ]:
for index, row in queries_df.iterrows():
    query = row['Query_Text']
    query_id = row['Query_Name']
    query_relevance_data = query_document_relevance_df[query_document_relevance_df['Query_Name'] == query_id]
    query_embedding = model.encode(query, convert_to_tensor=True)
    cosine_scores = util.pytorch_cos_sim(query_embedding, document_embeddings)[0]
    print(len(cosine_scores))
    top_k_scores, top_k_indices = torch.topk(cosine_scores, k=0)
    top_k_documents = [documents_df.loc[i, 'Document_Name'].replace(".txt", "") for i in top_k_indices.tolist()]
#     print(top_k_documents)
    actual_relevant_docs = query_relevance_data[query_relevance_data['Relevance'] == 1]['Document_Name'].tolist()
    print(actual_relevant_docs)
    num_actual = 0
    for doc in top_k_documents:
        if doc in actual_relevant_docs:
            num_actual += 1
    precision = num_actual/len(top_k_documents)
    recall = num_actual/len(actual_relevant_docs)
    precision_scores.append(precision)
    recall_scores.append(recall)
    

avg_precision = sum(precision_scores) / 50
avg_recall = sum(recall_scores) / 50

print(f"Average Precision: {avg_precision:.5f}")
print(f"Average Recall: {avg_recall:.5f}")